In [1]:
import json
import msgpack
from celery import Celery


MQ_IP = "52.10.216.28"

# Initialize Celery client to connect to the broker
app = Celery('tasks', broker=f'pyamqp://runner:UjLyQtRMWTG68aAKecq4Hn@{MQ_IP}//', backend='rpc://')

# Import the task module explicitly to ensure the task is registered
import comfyui_router.tasks
from comfyui_router.tasks import process_request
from comfyui_router.utils import extract_images_from_response


# JSON string to be sent
json_string = '{"6": {"inputs": {"text": "isekai joucho, 1girl, very long hair, best quality", "clip": ["101", 1]}, "class_type": "CLIPTextEncode", "_meta": {"title": "POSITIVE"}}, "7": {"inputs": {"text": "lowres, worst quality, displeasing, bad quality, bad anatomy, text, error, extra digit, cropped, average quality", "clip": ["101", 1]}, "class_type": "CLIPTextEncode", "_meta": {"title": "NEGATIVE"}}, "101": {"inputs": {"ckpt_name": "bxl-v5c-cont53/checkpoint-e1_s28000.safetensors"}, "class_type": "CheckpointLoaderSimple", "_meta": {"title": "Load Checkpoint"}}, "103": {"inputs": {"filename_prefix": "ComfyUI", "images": ["34:4", 0]}, "class_type": "SaveImage", "_meta": {"title": "Save Image"}}, "34:0": {"inputs": {"width": 896, "height": 1152, "batch_size": 4}, "class_type": "EmptyLatentImage", "_meta": {"title": "Empty Latent Image"}}, "34:1": {"inputs": {"multiplier": 0.75, "model": ["101", 0]}, "class_type": "RescaleCFG", "_meta": {"title": "RescaleCFG"}}, "34:2": {"inputs": {"sampling": "v_prediction", "zsnr": true, "model": ["34:1", 0]}, "class_type": "ModelSamplingDiscrete", "_meta": {"title": "ModelSamplingDiscrete"}}, "34:3": {"inputs": {"seed": 432737405573798, "steps": 28, "cfg": 5.5, "sampler_name": "euler", "scheduler": "normal", "denoise": 1, "model": ["34:2", 0], "positive": ["6", 0], "negative": ["7", 0], "latent_image": ["34:0", 0]}, "class_type": "KSampler", "_meta": {"title": "KSampler"}}, "34:4": {"inputs": {"samples": ["34:3", 0], "vae": ["101", 2]}, "class_type": "VAEDecode", "_meta": {"title": "VAE Decode"}}}'
json_data = json.loads(json_string)

# Unique key for the request
unique_key = "debug_request_001"

# Send the request to the Celery worker
result = app.send_task('comfyui_router.tasks.process_request', args=[unique_key, json_data])

# Wait for the result (blocking)
packed_response = result.get(timeout=40)  # You can adjust the timeout as needed

# Decode the MessagePack response
unique_key, raw_response = msgpack.unpackb(packed_response, raw=False)

node_imgs = extract_images_from_response(raw_response)
node_imgs

{'103': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=896x1152>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=896x1152>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=896x1152>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=896x1152>]}

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=896x1152>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=896x1152>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=896x1152>,
 <PIL.PngImagePlugin.PngImageFile image mode=RGB size=896x1152>]